In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [3]:
phi = pd.DataFrame(json.loads(open("./phi2_eval_df_c.json", "r").read()))
phi

,prompt,pred,actu,inf_time,temperature,difficulty,token_in,token_out,tokens_per_sec
0,### Task\nGenerate a SQL query to answer the f...,SELECT director FROM table_name_85 WHERE title...,SELECT director FROM table_name_85 WHERE title...,13.710582,0.2,easy,114,22,1.531664
1,### Task\nGenerate a SQL query to answer the f...,"SELECT 1971 FROM table_name_65 WHERE 1969 = ""30""","SELECT 1971 FROM table_name_65 WHERE 1969 = ""30""",11.643758,0.2,easy,103,19,1.545893
2,### Task\nGenerate a SQL query to answer the f...,SELECT team FROM table_name_73 WHERE kitmaker ...,SELECT team FROM table_name_73 WHERE kitmaker ...,17.942099,0.2,medium,129,34,1.839250
3,### Task\nGenerate a SQL query to answer the f...,SELECT result_score FROM table_name_45 WHERE r...,SELECT result_score FROM table_name_45 WHERE r...,13.449438,0.2,easy,112,23,1.635756
4,### Task\nGenerate a SQL query to answer the f...,SELECT COUNT(*) FROM pets AS T1 JOIN has_pet A...,SELECT COUNT(*) FROM student AS T1 JOIN has_pe...,23.890706,0.2,hard,147,51,2.092864
...,...,...,...,...,...,...,...,...,...
1253,### Task\nGenerate a SQL query to answer the f...,SELECT current_stock FROM table_17839_1 WHERE ...,SELECT current_stock FROM table_17839_1 WHERE ...,21.629273,0.2,easy,134,34,1.525710
1254,### Task\nGenerate a SQL query to answer the f...,SELECT T1.payment_date FROM payment AS T1 JOIN...,SELECT payment_date FROM payment WHERE amount ...,29.350447,0.2,hard,172,51,1.703552
1255,### Task\nGenerate a SQL query to answer the f...,SELECT venue FROM table_name_31 WHERE status =...,SELECT venue FROM table_name_31 WHERE status =...,21.359597,0.2,medium,126,35,1.591790
1256,### Task\nGenerate a SQL query to answer the f...,SELECT COUNT(rank) FROM table_name_27 WHERE re...,SELECT COUNT(rank) FROM table_name_27 WHERE re...,16.185911,0.2,easy,109,26,1.544553


In [ ]:
# Plot between time and difficulty
phi.groupby('difficulty')['tokens_per_sec'].mean().reset_index().plot()